In [20]:
import os
import urllib.parse as up
from SPARQLWrapper import SPARQLWrapper, TURTLE
import sys

ont_file = "address_ont.ttl"
data_file = "voies-paris-2023.csv"
query_file = "query.txt"
mapping_file = "mapping.json"
export_file = "addresses-temp.ttl"
out_file = "addresses.ttl"

export_format = "TURTLE"

abs_ont_file = os.path.abspath(ont_file)
abs_data_file = os.path.abspath(data_file)
abs_mapping_file = os.path.abspath(mapping_file)
abs_export_file = os.path.abspath(export_file)
abs_out_file = os.path.abspath(out_file)

project_name = "voies_paris_hist" # This repository must exist in graphdb

graphdb_url = "http://localhost:7200"
endpoint_url = "https://query.wikidata.org/sparql"

In [21]:
def get_query_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(TURTLE)
    return sparql.query().convert()

In [22]:
query = """
PREFIX addr: <http://rdf.geohistoricaldata.org/address#>

CONSTRUCT {
 ?street a addr:Landmark;
           addr:isLandmarkType addr:Thoroughfare;
           rdfs:label ?streetLabel;
           skos:altLabel ?streetAltLabel;
           addr:within ?loc.
  ?street addr:hasOfficialName [rdfs:label ?officialName; ]
}
WHERE {
  { ?street p:P361 [ps:P361 wd:Q16024163]. }
  UNION
  { ?street p:P361 [ps:P361 wd:Q107311481]. }
  ?street wdt:P131 ?loc.
  ?street rdfs:label ?streetLabel.
  FILTER (LANG(?streetLabel) = "fr")
  OPTIONAL {?street skos:altLabel ?streetAltLabel. FILTER (LANG(?streetAltLabel) = "fr")}
  OPTIONAL {
    ?street p:P1448 ?officialNameSt. 
    ?officialNameSt ps:P1448 ?officialName.
    OPTIONAL{?officialNameSt pq:P580 ?startDate}
    OPTIONAL{?officialNameSt pq:P582 ?endDate}
    FILTER (LANG(?officialName) = "fr")
    }
}
"""

In [23]:
get_query_results(endpoint_url, query).serialize(destination=abs_export_file)

/home/CBernard2/.local/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1179: RuntimeWarning: Format requested was TURTLE, but RDF/XML (application/rdf+xml;charset=utf-8) has been returned by the endpoint
  warnings.warn(


<Graph identifier=N193c947168ed4b6f917ec60b1320f84d (<class 'rdflib.graph.ConjunctiveGraph'>)>

In [24]:
# Launch GraphDB before launching this command
cmd1 = f"curl -X PUT -H \"Content-Type:application/rdf+xml\" {graphdb_url}/repositories/{project_name}"
cmd2 = f"curl -X DELETE -H \"Content-Type:application/x-turtle\" {graphdb_url}/repositories/{project_name}/statements"

os.system(cmd1)
os.system(cmd2)

REPOSITORY EXISTS: repository already exists: voies_paris_hist

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    62  100    62    0     0   3653      0 --:--:-- --:--:-- --:--:--  3875
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


0

In [25]:
# Launch GraphDB before launching this command
cmd1 = f"curl -X POST -H \"Content-Type:application/x-turtle\" -T \"{abs_export_file}\" {graphdb_url}/repositories/{project_name}/statements"
cmd2 = f"curl -X POST -H \"Content-Type:application/x-turtle\" -T \"{abs_ont_file}\" {graphdb_url}/repositories/{project_name}/statements"
os.system(cmd1)
os.system(cmd2)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 2894k    0     0  100 2894k      0  1834k  0:00:01  0:00:01 --:--:-- 1834k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19366    0     0  100 19366      0   163k --:--:-- --:--:-- --:--:--  163k


0